<img src="../images/JHI_STRAP_Web.png" style="width: 150px; float: right;">
# 07 - Programming for `UniProt`

## Table of Contents

1. [Introduction](#introduction)
2. [Python imports](#imports)
3. [Running a remote `UniProt` query](#uniprot)
  1. [Connecting to `UniProt`](#connect)
  2. [Constructing a query](#query)
  3. [Perform the query](#search)
  4. [EXAMPLE: Putting it together](#example01)
4. [Advanced queries](#advanced)
  1. [`key:value` queries](#keyvalue)
  2. [Exercise 01](#ex01)
  3. [Combining queries](#combine)
  4. [Exercise 02](#ex02)  
4. [Processing query results](#processing)
  1. [Tabular](#tabular)
  2. [Excel](#excel)  
  3. [FASTA sequence](#fasta)
  4. [`pandas` dataframe](#pandas)

<a id="introduction"></a>
## Introduction

The `UniProt` browser interface is very powerful, but you will have noticed that even the most complex queries can be converted into a single string that describes the search being made of the `UniProt` databases. This string is generated for you, and placed into the search field at the top of the `UniProt` webpage every time you run a search.

<p></p>
<div class="alert-success">
<b>It can be tedious and time-consuming to point-and-click your way through a large number of browser-based searches, but by using the `UniProt` webservice, the search strings you've already seen, and a Python module called `bioservices`, we can compose and run as many searches as we like using computer code, and pull the results of those searches.</b>
</div>

This notebook presents examples of methods for using `UniProt` programmatically, via a webservice, and you will be controlling the searches using Python code in this notebook.

There are a number of advantages to this approach:

* It is easy to set up repeatable searches for many sequences, or collections of sequences
* It is easy to read in the search results and conduct downstream analyses that add value to your search

Where it is not practical to submit a large number of simultaneous queries via a web form (because it is tiresome to point-and-click over and over again), this can be handled programmatically instead. You have the opportunity to change custom options to help refine your query, compared to the website interface. If you need to repeat a query, it can be trivial to get the same settings every time, if you use a programmatic approach.

<a id="imports"></a>
## Python imports

In [ ]:
# Show plots as part of the notebook
%matplotlib inline

# Standard library packages
import io

# Import Seaborn for graphics and plotting
import seaborn as sns

# Import bioservices module, to run remote UniProt queries
from bioservices import UniProt

# Import Pandas, so we can use dataframes
import pandas as pd

<a id="uniprot"></a>
## Running a remote `UniProt` query

<p></p>
<div class="alert-success">
<b>There are three key steps to running a remote `UniProt` query with `bioservices`:</b>
</div>

1. Make a link to the `UniProt` webservice
2. Construct a query string
3. Send the query to `UniProt`, and catch the result in a variable

Once the search result is caught, it can be processed in any way you like, written to a file, or ignored.

<a id="connect"></a>
### Connecting to `UniProt`

To open a connection to `UniProt`, you make an *instance* of the `UniProt()` *class* from `bioservices`. This can be persistent so that, once it is created, you can interact with it over and over again. 

<p></p>
<div class="alert-success">
<b>To make a persistent instance, you can assign `UniProt()` to a variable:</b>
</div>

```python
service = UniProt() # it is good practice to have a meaningful variable name
```

<a id="query"></a>
### Constructing a query

<p></p>
<div class="alert-success">
<b>`UniProt` allows for the construction of complex searches by combining *fields*. A full discussion is beyond the scope of this lesson, but you will have seen in [notebook 06](06-uniprot_browser.ipynb) that the searches you constructed by pointing and clicking on the `UniProt` website were converted into text in the search field at the top. </b>
</div>

To describe the format briefly: there are a set of defined *keys* - keywords that indicate the specific type of data you want to search in (such as `host`, `annotation`, or sequence `length`), and these are combined with a particular *value* you want to search for (such as `mouse`, or `40674`) in a `key:value` pair, separated by a colon, such as `host:mouse` or `ec:3.2.1.23`.

* `UniProt` query fields: [http://www.uniprot.org/help/query-fields](http://www.uniprot.org/help/query-fields)

If you provide a string, instead of a `key:value` pair, `UniProt` will search in all *fields* for your search term.

Programmatically, we construct the query as a *string*, e.g.

```python
query = "Q9AJE3"  # this query means we want to look in all fields for Q9AJE3
```

<a id="search"></a>
### Perform the query

To send the query to `UniProt`, you will use the `.search()` *method* of your active instance of the `UniProt()` *class*. 

<p></p>
<div class="alert-success">
<b>If you have assigned this instance to the variable `service` (as above), then you can run the `query` string as a remote search with the line:</b>
</div>

```python
result = service.search(query)  # Run a query and catch the output in result
```

In the line above, the output of the search (i.e. your results) are stored in a variable called `result`. It is good practice to make variable names short and descriptive - this makes your code easier to read.

<a id="example01"></a>
### EXAMPLE: Putting it together

The code in the cell below uses the example code above to create an instance of the `UniProt()` class, and use it to submit a pre-stored query to the `UniProt` service, then catch the result in a variable called `result`. The `print()` statement then shows us what the result looks like, as returned by the service.

In [ ]:
# Make a link to the UniProt webservice
service = UniProt()

# Build a query string
query = "Q9AJE3"

# Send the query to UniProt, and catch the search result in a variable
result = service.search(query)

# Inspect the result
print(result)

The `UniProt()` instance defined in the cell above is *persistent*, so you can reuse it to make another query, as in the cell below:

In [ ]:
# Make a new query string, and run a remote search at UniProt
new_query = "Q01844"
new_result = service.search(new_query)

# Inspect the result
print(new_result)

<a id="advanced"></a>
## Advanced queries

<p></p>
<div class="alert-success">
<b>The examples above built queries that were simple strings. They did not exploit the `key:value` search structure, or combine search terms. In this section, you will explore some queries that use the `UniProt` query fields, and combine them into powerful, filtering searches.</b>
</div>

<a id="keyvalue"></a>
### `key:value` queries

As noted above (and at [http://www.uniprot.org/help/query-fields](http://www.uniprot.org/help/query-fields)) particular values of specific data can be requested by using `key:value` pairs to restrict searches to named *fields* in the `UniProt` database.

As a first example, you will note that the result returned for the query `"Q01844"` has multiple entries. Only one of these is the sequence with `accession` value equal to `"Q01844"`, but the other entries make reference to this sequence somewhere in their database record. If we want to restrict our result only to the particular entry `"Q01844"`, we can specify the field we want to search as `accession`, and build the following query:

```python
query = "accession:Q01844"  # specify a search on the accession field
```

Note that we can use the same variable name `query` as earlier (this overwrites the previous value in `query`). The code below runs the search and shows the output:

In [ ]:
# Make a new query string, and run a remote search at UniProt
query = "accession:Q01844"
result = service.search(query)

# Inspect the result
print(result)

<div class="alert-success">
<b>By using this and other `key:value` constructions, we can refine our searches to give us only the entries we're interested in</b>
</div>

<img src="../images/exercise.png" style="width: 100px; float: left;">
<a id="ex01"></a>
### Exercise 01 (10min)

Using `key:value` searches, can you find and download sets of entries for proteins that satisfy the following requirements? (**HINT:** this <a href="http://www.uniprot.org/help/query-fields">link</a> to the `UniProt` query fields may be helpful, here):

<p></p>
<div class="alert-danger">
<ul>
<li> Have publications authored by someone with the surname Broadhurst
<li> Have protein length between 9000aa and 9010aa
<li> Derive from the taipan snake
<li> Have been found in the wing
</ul>
</div>

In [ ]:
# SOLUTION - EXERCISE 01
queries = ["author:broadhurst", "length:[9000 TO 9010]", "organism:taipan", "tissue:wing"]

for query in queries:
    print("\n%s" % query)
    print("=" * len(query))
    result = service.search(query)
    print(result)

<a id="combine"></a>
### Combining queries

<p></p>
<div class="alert-success">
<b>Combining terms in a `UniProt` query can be as straightforward as putting them in the same string, separated by a space.</b>
</div>

For example:

```python
query = "organism:rabbit tissue:eye"
```

will search for all entries deriving from rabbits that are found in the eye

In [ ]:
# Combine queries for rabbit (organism) and eye (tissue), and search
query = "organism:rabbit tissue:eye"
result = service.search(query)
print(result)

<img src="../images/exercise.png" style="width: 100px; float: left;">
<a id="ex02"></a>
### Exercise 02 (10min)

Using `key:value` searches, can you find and download sets of entries for proteins that satisfy the following requirements? (**HINT:** this <a href="http://www.uniprot.org/help/query-fields">link</a> to the `UniProt` query fields may be helpful, here):

<p></p>
<div class="alert-danger">
<ul>
<li> Found in sheep spleen
<li> Have "rxlr" in their name, have a publication with author name Pritchard, and are between 70aa and 80aa in length
<li> Derive from a quokka and have had their annotations manually reviewed
<li> Are found in cell membranes of horse heart tissue, and have had their annotations manually reviewed
</ul>
</div>

In [ ]:
# SOLUTION - EXERCISE 02
queries = ["organism:sheep tissue:spleen",
           "author:pritchard name:rxlr length:[70 TO 80]",
           "organism:quokka reviewed:yes",
           "go:membrane organism:horse tissue:heart reviewed:yes"]

for query in queries:
    print("\n%s" % query)
    print("=" * len(query))
    result = service.search(query)
    print(result)

<a id="boolean"></a>
### Combining terms with Boolean logic

Boolean logic allows you to combine search terms with each other in arbitrary ways using three *operators*, specifying whether:

* both terms are required (`AND`) **NOTE:** this is implicitly what you have been doing in the examples above
* either term is allowed (`OR`)
* a term is disallowed (`NOT`)

Searches are read from left-to right, but the logic of a search can be controlled by placing the combinations you want to resolve first in parentheses (`()`). Combining these operators can build some extremely powerful searches. For example, to get all proteins from horses and sheep, identified in the ovary, and having length greater than 200aa, you could use the query:

```
query = "tissue:ovary AND (organism:sheep OR organism:horse) NOT length:[1 TO 200]"
```

<a id="processing"></a>
## Processing query results

So far you have worked with the default output from `bioservices`, although you know from [`notebook 06`](06-uniprot_browser.ipynb) that `UniProt` can provide output in a number of useful formats for searches in the browser.

The default output is `tabular`, and gives a good idea of the nature and content of the entries you recover. In this section, you will see some ways to download search results in alternative formats, which can be useful for analysis.

All the output format options are controlled in a similar way, using the `frmt=<format>` argument when you conduct your search - with `<format>` being one of the allowed terms (see the [`bioservices` documentation](https://pythonhosted.org/bioservices/references.html#module-bioservices.uniprot) for a full list.

<a id="tabular"></a>
### Tabular

<p></p>
<div class="alert-success">
<b>The default datatype is the most flexible datatype for download: <i>tabular</i>.</b>
</div>

This can be specified explicitly with the `tab` format:

```python
result = service.search(query, frmt="tab")
```

In [ ]:
# Make a query string, and run a remote search at UniProt,
# getting the result as tabular format
query = "Q01844"
result = service.search(query, frmt="tab")

# Inspect the result
print(result)

By default, the columns that are returned are: `Entry`, `Entry name`, `Status`, `Protein names`, `Gene names`, `Organism`, and `Length`. But these can be modified by passing the `columns=<list>` argument, where the `<list>` is a comma-separated list of column names. For example:

```python
columnlist = "id,entry name,length,organism,mass,domains,domain,pathway"
result = service.search(query, frmt="tab", columns=columnlist)
```

The list of allowed column names can be found by inspecting the content of the variable `service._valid_columns`.

In [ ]:
# Make a query string
query = "Q01844"

# Define a list of columns we want to retrive
columnlist = "id,entry name,length,mass,go(cellular component)"

# Run the remote search
result = service.search(query, frmt="tab", columns=columnlist)

# View the result
print(result)

#### Converting to a dataframe

<p></p>
<div class="alert-success">
<b>The `pandas` module allows us to process tabular data into dataframes, just like in `R`.</b>
</div>

To do this, we have to use the `io.StringIO()` class to make it think that our downloaded results are a file

```python
df = pd.read_table(io.StringIO(result))
```

In [ ]:
# Convert the last saerch result into a dataframe in Pandas
df = pd.read_table(io.StringIO(result))

# View the dataframe
df

Doing this will produce a `pandas` dataframe that can be manipulated and analysed just like any other dataframe. We can, for instance, view a histogram of sequence lengths from the table above:

In [ ]:
# Plot histogram from dataframe
df.hist();

<a id="excel"></a>
### Excel

<p></p>
<div class="alert-success">
<b>You can download Excel spreadsheets directly from `UniProt`, just as with the browser interface.</b>
</div>

```python
result = service.search(query, frmt="xls")
```

You can't use the Excel output directly in your code without some file manipulation, and you'll have to save it to a file, as in the example below. Also, the downloaded format is not guaranteed to be current for your version of Excel, and the application may ask to repair it. But, if you want Excel output to share with/display to others, you can get it programmatically.

<p></p>
<div class="alert-danger">
<b>NOTE: the downloaded format is actually `.xlsx`, rather than `.xls` which is implied by the format</b>
</div>

In [ ]:
# Make a query string, and run a remote search at UniProt,
# getting the result as an Excel spreadsheer
query = "Q01844"
result = service.search(query, frmt="xls")

# Write the Excel spreadsheet to file
outfile = 'output/uniprot/Q01844.xlsx'
with open(outfile, 'wb') as ofh:
    ofh.write(result)

<a id="fasta"></a>
### FASTA sequence

<p></p>
<div class="alert-success">
<b>If you're interested only in the FASTA format sequence for an entry, you can use the `fasta` option with `frmt` to recover the sequences directly, as in the example below:</b>
</div>

In [ ]:
# Make a query string, and run a remote search at UniProt,
# getting the result as FASTA sequence
query = "go:membrane organism:horse tissue:heart reviewed:yes"
result = service.search(query, frmt="fasta")

# Inspect the result
print(result)

<a id="pandas"></a>
### `pandas` dataframe

<p></p>
<div class="alert-success">
<b>In addition to the conversion of tabular output to a `pandas` dataframe above, you can ask the `UniProt()` instance to return a `pandas` dataframe directly, with the `.get_df()` method.</b>
</div>

```python
result = service.get_df(query)
```

For a single ID, it's possible to pass the ID directly, for example:

In [ ]:
# Get a dataframe for a single accession ID
df = service.get_df('tissue:venom (organism:snake OR organism:mamba)')

# View the dataframe
df.head()

This dataframe works like any other dataframe. You can get a complete list of returned columns:

In [ ]:
df.columns

and use the convenient features of a dataframe, such as built-in plotting:

In [ ]:
# Construct a histogram of returned sequence lengths
df.hist('Length', bins=100);

and grouping/subsetting:

In [ ]:
# Subset out coral snakes
coral = df.loc[df["Organism"].str.contains("coral")]

# Plot a strip plot of sequence size by organism in the dataframe
output = sns.stripplot(y="Organism", x="Length",
                       data=coral)  # Render strip plot

<img src="../images/exercise.png" style="width: 100px; float: left;">
<a id="ex03"></a>
### Exercise 03 (10min)

Can you use `bioservices`, `UniProt` and `pandas` to:

<p></p>
<div class="alert-danger">
<ul>
<li> download a dataframe for all proteins that have "rxlr" in their name
<li> render a violin plot (`sns.violinplot()`) that shows the distribution of protein lengths grouped according to the evidence for the protein
</ul>
</div>

In [ ]:
# SOLUTION - EXERCISE 03

# Get dataframe
df = service.get_df('name:rxlr')

# Draw violin plot
output = sns.violinplot(y="Protein existence", x="Length", data=df)